In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import datetime as dt
import math
from sklearn.preprocessing import StandardScaler
import helper
from helper import *

In [2]:
df = load_and_preprocess()
df = df.reset_index(drop = True)

In [3]:
df.shape

(45252, 16)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 0 to 45251
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        45252 non-null  object 
 1   Description     45252 non-null  object 
 2   Humidity        45252 non-null  float64
 3   Wind Direction  45252 non-null  float64
 4   Temperature     45252 non-null  float64
 5   Pressure        45252 non-null  float64
 6   Wind Speed      45252 non-null  float64
 7   clouds          45252 non-null  int64  
 8   rain            45252 non-null  int64  
 9   mist            45252 non-null  int64  
 10  snow            45252 non-null  int64  
 11  shower          45252 non-null  int64  
 12  thunderstorm    45252 non-null  int64  
 13  fog             45252 non-null  int64  
 14  other           45252 non-null  int64  
 15  Intensity       45252 non-null  float64
dtypes: float64(6), int64(8), object(2)
memory usage: 5.5+ MB


In [5]:
df['datetime']= pd.to_datetime(df['datetime'])

#### Just Using Date/Previous Hour Temp.

In [6]:
df.head()

,datetime,Description,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,thunderstorm,fog,other,Intensity
0,2012-10-01 13:00:00,heavy clouds,93.0,1001.0,285.830000,230.0,4.0,1,0,0,0,0,0,0,0,3.0
1,2012-10-01 14:00:00,sky is clear,91.0,986.0,285.834650,230.0,4.0,0,0,0,0,0,0,0,0,0.0
2,2012-10-01 15:00:00,sky is clear,87.0,945.0,285.847790,231.0,4.0,0,0,0,0,0,0,0,0,0.0
3,2012-10-01 16:00:00,sky is clear,84.0,904.0,285.860929,233.0,4.0,0,0,0,0,0,0,0,0,0.0
4,2012-10-01 17:00:00,sky is clear,80.0,863.0,285.874069,234.0,3.0,0,0,0,0,0,0,0,0,0.0


In [7]:
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year

#lag features from Duncan
df['lag1'] = df['Temperature'].shift(periods = 1, fill_value = 0)
df['lag2'] = df['Temperature'].shift(periods = 2, fill_value = 0)
df['lag12'] = df['Temperature'].shift(periods = 12, fill_value = 0)
df['lag30'] = df['Temperature'].shift(periods = 30, fill_value = 0)

df['avg_Temperature'] = 0 
for i in range(1, len(df)):
    df['avg_Temperature'][i] = df['lag1'][:i+1].mean()
    

df.drop('Description', axis = 1, inplace = True)
df = df.set_index('datetime')

#we have to eliminate all those with 0s
df = df[df['lag30'] != 0]
df

C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Humidity,Wind Direction,Temperature,Pressure,Wind Speed,clouds,rain,mist,snow,shower,...,other,Intensity,day,month,year,lag1,lag2,lag12,lag30,avg_Temperature
datetime,,,,,,,,,,,,,,,,,,,,,
2012-10-02 19:00:00,71.861538,1016.424953,288.94,282.0,2.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.94,288.69,286.058023,285.830000,277
2012-10-02 20:00:00,71.861538,1016.424953,288.69,239.0,5.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.94,288.94,286.071162,285.834650,277
2012-10-02 21:00:00,77.000000,1007.000000,288.55,260.0,7.0,1,0,0,0,0,...,0,3.0,2,10,2012,288.69,288.94,286.084302,285.847790,277
2012-10-02 22:00:00,71.861538,1016.424953,288.22,284.0,0.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.55,288.69,286.097442,285.860929,278
2012-10-02 23:00:00,71.861538,1016.424953,287.92,267.0,0.0,0,0,0,0,0,...,0,0.0,2,10,2012,288.22,288.55,286.110581,285.874069,278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-29 20:00:00,64.000000,1021.000000,275.13,300.0,10.0,1,0,0,0,0,...,0,2.0,29,11,2017,276.45,276.15,275.380000,265.120000,280
2017-11-29 21:00:00,59.000000,1023.000000,274.13,300.0,7.0,1,0,0,0,0,...,0,2.0,29,11,2017,275.13,276.45,277.450000,266.120000,280
2017-11-29 22:00:00,66.000000,1024.000000,273.48,290.0,7.0,1,0,0,0,0,...,0,1.0,29,11,2017,274.13,275.13,278.770000,268.100000,280


# Causal Inference

In [8]:
import dowhy
from dowhy import CausalModel

In [9]:
features = df.drop('Temperature', axis=1)
cols = list(features.columns)
cols

['Humidity',
 'Wind Direction',
 'Pressure',
 'Wind Speed',
 'clouds',
 'rain',
 'mist',
 'snow',
 'shower',
 'thunderstorm',
 'fog',
 'other',
 'Intensity',
 'day',
 'month',
 'year',
 'lag1',
 'lag2',
 'lag12',
 'lag30',
 'avg_Temperature']

In [10]:
X = df[cols]
effects = dict()
test = dict()

for i in X:
    model = CausalModel(data = df,
                        treatment = i,
                        outcome = 'Temperature',
                        common_causes = list(X.drop(i, axis = 'columns').columns))
    
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

    estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.linear_regression",
                                    test_significance = True)
    
    effects[i] = estimate.value
    test[i] = estimate

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['Humidity'] on outcome ['Temperature']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:b: Temperature~Humidity+Wind Speed+lag2+shower+lag1+lag30+fog+thunderstorm+day+other+lag12+rain+month+Pressure+snow+clouds+Intensity+Wind Direction+avg_Temperature+mist+year
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['snow'] on outcome ['Temperature']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:b: Temperature~snow+Wind Speed+lag2+shower+lag1+lag30+fog+thunderstorm+day+other+lag12+rain+month+Pressure+clouds+Intensity+Wind Direction+Humidity+avg_Temperature+mist+year
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['shower'] on outcome ['Temperature']
INFO:dowhy.causal_identifier:Continuin

INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:b: Temperature~month+Wind Speed+lag2+shower+lag1+lag30+fog+thunderstorm+day+other+lag12+rain+Pressure+snow+clouds+Intensity+Wind Direction+Humidity+avg_Temperature+mist+year
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['year'] on outcome ['Temperature']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.cau

## Final Output Preprocessing

In [11]:
#piecing together the dataframes and also processing out the p-values (brute force)
estimates = pd.DataFrame(effects.items(),columns=['Feature', 'Estimate'])
tests = pd.DataFrame(test.items(), columns=['Feature', 'P-Values'])
tests = tests.astype(str)
tests['P-Values'] = tests['P-Values'].str.split('p-value: ').str[-1]
tests['P-Values'] = tests['P-Values'].str.replace(r'[', '')
tests['P-Values'] = tests['P-Values'].str.replace(r']\n', '')
tests = tests.astype({"P-Values": float})
tests['Estimate'] = estimates['Estimate']
finaldf = tests.round(3).sort_values('Estimate')

C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
C:\Users\Steven\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


In [12]:
finaldf 

,Feature,P-Values,Estimate
9,thunderstorm,0.000,-0.510
17,lag2,0.000,-0.262
8,shower,0.046,-0.062
19,lag30,0.000,-0.031
3,Wind Speed,0.005,-0.007
0,Humidity,0.000,-0.003
2,Pressure,0.000,-0.001
13,day,0.883,0.000
1,Wind Direction,0.038,0.001
20,avg_Temperature,0.601,0.002


In [13]:
#features with high causal inference
high_evd = finaldf[finaldf['P-Values'] <= 0.05]
high_evd

,Feature,P-Values,Estimate
9,thunderstorm,0.000,-0.510
17,lag2,0.000,-0.262
8,shower,0.046,-0.062
19,lag30,0.000,-0.031
3,Wind Speed,0.005,-0.007
0,Humidity,0.000,-0.003
2,Pressure,0.000,-0.001
1,Wind Direction,0.038,0.001
14,month,0.000,0.008
18,lag12,0.000,0.035


In [14]:
#features with low causal inference
low_evd = finaldf[finaldf['P-Values'] >= 0.05]
low_evd

,Feature,P-Values,Estimate
13,day,0.883,0.000
20,avg_Temperature,0.601,0.002
15,year,0.500,0.003
12,Intensity,0.268,0.010
10,fog,0.664,0.026
11,other,0.641,0.119
